In [27]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [28]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime

headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0' }
base_url = 'https://www.glyndebourne.com/about-us/our-archive/performance-archive/PerformanceSearchForm?Year=&Name=&action_doPerformanceSearch=Search&Opera='

In [29]:
listings = []

url = 'https://www.glyndebourne.com/about-us/our-archive/performance-archive/PerformanceSearchForm?Year=&Name=&action_doPerformanceSearch=Search&Opera=1'
r  = requests.get(url, headers=headers)
soup = BeautifulSoup(r.text, "html5lib")
options = soup.find('select', attrs={'id': 'Form_PerformanceSearchForm_Opera'}).find_all('option')

for option in options:
    
    work = option.get_text()
    work_id = option.get('value')
    
    url = base_url + str(work_id)
    r  = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, "html5lib")
    
    results = soup.find('div', attrs={'class': 'unit-archive__search-results'}).find_all('a')
    for result in results:
        result_url = 'https://www.glyndebourne.com' + result.get('href')
        r  = requests.get(result_url, headers=headers)
        soup = BeautifulSoup(r.text, "html5lib")
        
        composer = ''
        details = soup.find('div', attrs={'class': 'performance-details'}).find_all('p')
        for detail in details:
            if detail.find('strong').get_text().strip() == 'Composer:':
                composer = detail.find('strong').next_sibling.strip()
        
        performances = soup.find_all('article', attrs={'class': 'unit-archive__season_performance'})    
        for performance in performances:
            date = performance.find('strong').get_text()
            date = datetime.strptime(date, '%d %B %Y').date()                    
            listing = []
            listing.append(work)                        
            listing.append(composer)
            listing.append(date)
            print(listing)
            listings.append(listing)

#Saving results into dataframe and folder            
import pandas as pd                     
headers = ['work', 'composer', 'date']
df = pd.DataFrame(listings, columns=headers)
df.to_csv('../../data/processed/listings/glyndebourne.csv')                            

["A Midsummer Night's Dream", 'Benjamin Britten', datetime.date(1981, 6, 21)]
["A Midsummer Night's Dream", 'Benjamin Britten', datetime.date(1981, 6, 23)]
["A Midsummer Night's Dream", 'Benjamin Britten', datetime.date(1981, 6, 25)]
["A Midsummer Night's Dream", 'Benjamin Britten', datetime.date(1981, 6, 27)]
["A Midsummer Night's Dream", 'Benjamin Britten', datetime.date(1981, 7, 1)]
["A Midsummer Night's Dream", 'Benjamin Britten', datetime.date(1981, 7, 3)]
["A Midsummer Night's Dream", 'Benjamin Britten', datetime.date(1981, 7, 5)]
["A Midsummer Night's Dream", 'Benjamin Britten', datetime.date(1981, 7, 7)]
["A Midsummer Night's Dream", 'Benjamin Britten', datetime.date(1981, 7, 10)]
["A Midsummer Night's Dream", 'Benjamin Britten', datetime.date(1981, 7, 12)]
["A Midsummer Night's Dream", 'Benjamin Britten', datetime.date(1981, 8, 24)]
["A Midsummer Night's Dream", 'Benjamin Britten', datetime.date(1981, 9, 26)]
["A Midsummer Night's Dream", 'Benjamin Britten', datetime.date(1981

['Albert Herring', 'Benjamin Britten', datetime.date(1986, 10, 10)]
['Albert Herring', 'Benjamin Britten', datetime.date(1986, 10, 11)]
['Albert Herring', 'Benjamin Britten', datetime.date(1986, 10, 18)]
['Albert Herring', 'Benjamin Britten', datetime.date(1986, 10, 25)]
['Albert Herring', 'Benjamin Britten', datetime.date(1986, 11, 1)]
['Albert Herring', 'Benjamin Britten', datetime.date(1986, 11, 8)]
['Albert Herring', 'Benjamin Britten', datetime.date(1986, 11, 15)]
['Albert Herring', 'Benjamin Britten', datetime.date(1986, 8, 16)]
['Albert Herring', 'Benjamin Britten', datetime.date(1989, 2, 4)]
['Albert Herring', 'Benjamin Britten', datetime.date(1989, 2, 5)]
['Albert Herring', 'Benjamin Britten', datetime.date(1989, 2, 1)]
['Albert Herring', 'Benjamin Britten', datetime.date(1990, 5, 24)]
['Albert Herring', 'Benjamin Britten', datetime.date(1990, 5, 26)]
['Albert Herring', 'Benjamin Britten', datetime.date(1990, 5, 29)]
['Albert Herring', 'Benjamin Britten', datetime.date(1990, 6

['Arabella', 'Richard Strauss', datetime.date(1996, 6, 23)]
['Arabella', 'Richard Strauss', datetime.date(1996, 6, 28)]
['Arabella', 'Richard Strauss', datetime.date(1996, 7, 1)]
['Arabella', 'Richard Strauss', datetime.date(1996, 7, 7)]
['Arabella', 'Richard Strauss', datetime.date(1996, 7, 11)]
['Arabella', 'Richard Strauss', datetime.date(1996, 7, 14)]
['Arabella', 'Richard Strauss', datetime.date(1996, 7, 19)]
['Arabella', 'Richard Strauss', datetime.date(1996, 7, 22)]
['Arabella', 'Richard Strauss', datetime.date(1996, 7, 26)]
['Arabella', 'Richard Strauss', datetime.date(1996, 8, 4)]
['Arabella', 'Richard Strauss', datetime.date(1996, 8, 8)]
['Arabella', 'Richard Strauss', datetime.date(1996, 8, 11)]
['Arabella', 'Richard Strauss', datetime.date(1996, 8, 14)]
['Arabella', 'Richard Strauss', datetime.date(1996, 8, 18)]
['Arabella', 'Richard Strauss', datetime.date(1996, 8, 24)]
['Ariadne auf Naxos (1st version)', 'Richard Strauss', datetime.date(1950, 8, 21)]
['Ariadne auf Naxos (

['Ariadne auf Naxos (2nd version)', 'Richard Strauss', datetime.date(1981, 8, 13)]
['Ariadne auf Naxos (2nd version)', 'Richard Strauss', datetime.date(2013, 5, 18)]
['Ariadne auf Naxos (2nd version)', 'Richard Strauss', datetime.date(2013, 5, 22)]
['Ariadne auf Naxos (2nd version)', 'Richard Strauss', datetime.date(2013, 5, 25)]
['Ariadne auf Naxos (2nd version)', 'Richard Strauss', datetime.date(2013, 5, 30)]
['Ariadne auf Naxos (2nd version)', 'Richard Strauss', datetime.date(2013, 6, 2)]
['Ariadne auf Naxos (2nd version)', 'Richard Strauss', datetime.date(2013, 6, 4)]
['Ariadne auf Naxos (2nd version)', 'Richard Strauss', datetime.date(2013, 6, 7)]
['Ariadne auf Naxos (2nd version)', 'Richard Strauss', datetime.date(2013, 6, 13)]
['Ariadne auf Naxos (2nd version)', 'Richard Strauss', datetime.date(2013, 6, 16)]
['Ariadne auf Naxos (2nd version)', 'Richard Strauss', datetime.date(2013, 6, 20)]
['Ariadne auf Naxos (2nd version)', 'Richard Strauss', datetime.date(2013, 6, 23)]
['Ariad

['Capriccio', 'Richard Strauss', datetime.date(1976, 6, 29)]
['Capriccio', 'Richard Strauss', datetime.date(1976, 7, 2)]
['Capriccio', 'Richard Strauss', datetime.date(1976, 7, 7)]
['Capriccio', 'Richard Strauss', datetime.date(1976, 7, 9)]
['Capriccio', 'Richard Strauss', datetime.date(1976, 7, 11)]
['Capriccio', 'Richard Strauss', datetime.date(1976, 7, 13)]
['Capriccio', 'Richard Strauss', datetime.date(1976, 7, 16)]
['Capriccio', 'Richard Strauss', datetime.date(1976, 7, 21)]
['Capriccio', 'Richard Strauss', datetime.date(1976, 7, 25)]
['Capriccio', 'Richard Strauss', datetime.date(1976, 9, 23)]
['Capriccio', 'Richard Strauss', datetime.date(1976, 9, 30)]
['Capriccio', 'Richard Strauss', datetime.date(1976, 10, 7)]
['Capriccio', 'Richard Strauss', datetime.date(1976, 10, 14)]
['Capriccio', 'Richard Strauss', datetime.date(1976, 10, 21)]
['Capriccio', 'Richard Strauss', datetime.date(1976, 8, 20)]
['Capriccio', 'Richard Strauss', datetime.date(1976, 9, 16)]
['Capriccio', 'Richard St

...

['The Makropulos Case', 'Leos Janacek', datetime.date(2001, 6, 29)]
['The Makropulos Case', 'Leos Janacek', datetime.date(2001, 7, 5)]
['The Makropulos Case', 'Leos Janacek', datetime.date(2001, 7, 9)]
['The Makropulos Case', 'Leos Janacek', datetime.date(2001, 7, 14)]
['The Makropulos Case', 'Leos Janacek', datetime.date(2001, 7, 22)]
['The Makropulos Case', 'Leos Janacek', datetime.date(2001, 7, 26)]
['The Makropulos Case', 'Leos Janacek', datetime.date(2001, 7, 28)]
['The Makropulos Case', 'Leos Janacek', datetime.date(2001, 7, 30)]
['The Makropulos Case', 'Leos Janacek', datetime.date(2001, 8, 5)]
['The Makropulos Case', 'Leos Janacek', datetime.date(2001, 8, 9)]
['The Makropulos Case', 'Leos Janacek', datetime.date(2001, 8, 13)]
['The Makropulos Case', 'Leos Janacek', datetime.date(2001, 8, 19)]
['The Makropulos Case', 'Leos Janacek', datetime.date(2001, 8, 22)]
['The Makropulos Case', 'Leos Janacek', datetime.date(2001, 2, 11)]
['The Makropulos Case', 'Leos Janacek', datetime.dat

["The Rake's Progress", 'Igor Stravinsky', datetime.date(1975, 9, 19)]
["The Rake's Progress", 'Igor Stravinsky', datetime.date(1975, 9, 25)]
["The Rake's Progress", 'Igor Stravinsky', datetime.date(1975, 10, 2)]
["The Rake's Progress", 'Igor Stravinsky', datetime.date(1975, 10, 9)]
["The Rake's Progress", 'Igor Stravinsky', datetime.date(1975, 10, 16)]
["The Rake's Progress", 'Igor Stravinsky', datetime.date(1975, 10, 23)]
["The Rake's Progress", 'Igor Stravinsky', datetime.date(1975, 8, 7)]
["The Rake's Progress", 'Igor Stravinsky', datetime.date(1975, 6, 21)]
["The Rake's Progress", 'Igor Stravinsky', datetime.date(1975, 6, 26)]
["The Rake's Progress", 'Igor Stravinsky', datetime.date(1975, 6, 28)]
["The Rake's Progress", 'Igor Stravinsky', datetime.date(1975, 6, 30)]
["The Rake's Progress", 'Igor Stravinsky', datetime.date(1975, 7, 3)]
["The Rake's Progress", 'Igor Stravinsky', datetime.date(1975, 7, 5)]
["The Rake's Progress", 'Igor Stravinsky', datetime.date(1975, 7, 7)]
["The Ra

['The Rape of Lucretia', 'Benjamin Britten', datetime.date(1946, 7, 29)]
['The Rape of Lucretia', 'Benjamin Britten', datetime.date(1946, 7, 30)]
['The Rape of Lucretia', 'Benjamin Britten', datetime.date(1946, 7, 31)]
['The Rape of Lucretia', 'Benjamin Britten', datetime.date(1946, 8, 1)]
['The Rape of Lucretia', 'Benjamin Britten', datetime.date(1946, 8, 2)]
['The Rape of Lucretia', 'Benjamin Britten', datetime.date(1946, 8, 3)]
['The Rape of Lucretia', 'Benjamin Britten', datetime.date(1946, 8, 3)]
['The Rape of Lucretia', 'Benjamin Britten', datetime.date(1946, 8, 5)]
['The Rape of Lucretia', 'Benjamin Britten', datetime.date(1946, 8, 6)]
['The Rape of Lucretia', 'Benjamin Britten', datetime.date(1946, 8, 7)]
['The Rape of Lucretia', 'Benjamin Britten', datetime.date(1946, 8, 8)]
['The Rape of Lucretia', 'Benjamin Britten', datetime.date(1946, 8, 9)]
['The Rape of Lucretia', 'Benjamin Britten', datetime.date(1946, 8, 10)]
['The Rape of Lucretia', 'Benjamin Britten', datetime.date(1

["The Soldier's Tale", 'Igor Stravinsky', datetime.date(1954, 9, 6)]
["The Soldier's Tale", 'Igor Stravinsky', datetime.date(1954, 9, 7)]
["The Soldier's Tale", 'Igor Stravinsky', datetime.date(1954, 9, 9)]
["The Soldier's Tale", 'Igor Stravinsky', datetime.date(1954, 9, 10)]
['The Turn of the Screw', 'Benjamin Britten', datetime.date(2006, 10, 18)]
['The Turn of the Screw', 'Benjamin Britten', datetime.date(2006, 10, 21)]
['The Turn of the Screw', 'Benjamin Britten', datetime.date(2006, 10, 24)]
['The Turn of the Screw', 'Benjamin Britten', datetime.date(2006, 10, 27)]
['The Turn of the Screw', 'Benjamin Britten', datetime.date(2006, 11, 2)]
['The Turn of the Screw', 'Benjamin Britten', datetime.date(2006, 11, 9)]
['The Turn of the Screw', 'Benjamin Britten', datetime.date(2006, 11, 16)]
['The Turn of the Screw', 'Benjamin Britten', datetime.date(2006, 11, 23)]
['The Turn of the Screw', 'Benjamin Britten', datetime.date(2006, 11, 30)]
['The Turn of the Screw', 'Benjamin Britten', date

['Tristan und Isolde', 'Richard Wagner', datetime.date(2007, 8, 1)]
['Tristan und Isolde', 'Richard Wagner', datetime.date(2007, 8, 6)]
['Tristan und Isolde', 'Richard Wagner', datetime.date(2007, 8, 10)]
['Tristan und Isolde', 'Richard Wagner', datetime.date(2007, 8, 14)]
['Tristan und Isolde', 'Richard Wagner', datetime.date(2007, 8, 18)]
['Tristan und Isolde', 'Richard Wagner', datetime.date(2007, 8, 22)]
['Tristan und Isolde', 'Richard Wagner', datetime.date(2009, 8, 6)]
['Tristan und Isolde', 'Richard Wagner', datetime.date(2009, 8, 10)]
['Tristan und Isolde', 'Richard Wagner', datetime.date(2009, 8, 14)]
['Tristan und Isolde', 'Richard Wagner', datetime.date(2009, 8, 18)]
['Tristan und Isolde', 'Richard Wagner', datetime.date(2009, 8, 22)]
['Tristan und Isolde', 'Richard Wagner', datetime.date(2009, 8, 26)]
['Tristan und Isolde', 'Richard Wagner', datetime.date(2009, 8, 30)]
['Un Ballo in Maschera', 'Giuseppe Verdi', datetime.date(1949, 8, 22)]
['Un Ballo in Maschera', 'Giuseppe 